In [2]:
from bs4 import BeautifulSoup
import re
import requests as req
import pandas as pd
import json
#import ijson

In [4]:
baseurl="https://theanalyst.com/competition/premier-league/stats?utm_source="

In [6]:
def askurl(url):
        headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}
        try:
            response = req.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            html =response.text
            with open('football.html', 'w', encoding='utf-8') as file:
                file.write(html)
            #print(ltml)
            return html
        except req.exceptions.HTTPError as e:
            print(f"HTTP error occurred:{e.response.status_code}-{e.response.reason}")
        except req.exceptions.RequestException as e:
            print(f"Requset error occurred:{e}")
        return ""

In [8]:
def json_from_html(url):
    html = askurl(url)     # 保存获取到的网页源码
    soup = BeautifulSoup(html, "html.parser")      # 使用 html.parser 解析 HTML
    json_tag = soup.find("script",{"class":"data","type":"application/json"})  #提取html中包含球员数据的json文件
    #提取解析json
    if json_tag:
        try:
            json_data = json.loads(json_tag.string)
            with open("player.json","w",encoding="utf-8") as file:
                json.dump(json_data, file, ensure_ascii=False, indent=4)
            print("json已经保存")
            return True
        except json.JSONDecodeError as e:
            print(f"解析失败:{e}")
            return False
    else:
        print("没有找到json")
        return False

In [61]:
askurl(baseurl)

'  <!doctype html>\n<html class="" lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge"><script type="text/javascript">(window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:["bam.nr-data.net"]},distributed_tracing:{enabled:true}};(window.NREUM||(NREUM={})).loader_config={agentID:"1588640526",accountID:"3422235",trustKey:"66686",xpid:"VwIFU1RQDRABUVNQAQUOUlcA",licenseKey:"NRJS-034728da04015584270",applicationID:"1522649431"};;/*! For license information please see nr-loader-spa-1.277.0.min.js.LICENSE.txt */\n(()=>{var e,t,r={8122:(e,t,r)=>{"use strict";r.d(t,{a:()=>i});var n=r(944);function i(e,t){try{if(!e||"object"!=typeof e)return(0,n.R)(3);if(!t||"object"!=typeof t)return(0,n.R)(4);const r=Object.create(Object.getPrototypeOf(t),Object.getOwnPropertyDescriptors(t)),o=0===Object.keys(r).length?e:r;for(let a in o)if(void 0!==e[a])try{if(

In [63]:
json_from_html(baseurl)

json已经保存


True

In [10]:
import json

def inspect_player_fields(json_file):
    try:
        # 打开并读取 JSON 文件
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # 检查是否存在 'player' 字段
        if "player" not in data:
            print("'player' 字段未找到！")
            return
        
        # 获取 player 数据
        player_data = data["player"]

        # 打印 player 的所有键
        if isinstance(player_data, dict):  # 如果 player 是一个字典
            print("player 字段下的所有键：")
            for key in player_data.keys():
                print(f"- {key}")
        else:
            print(f"'player' 字段不是字典类型，而是 {type(player_data)} 类型")

    except FileNotFoundError:
        print(f"文件 {json_file} 未找到，请检查文件路径。")
    except json.JSONDecodeError:
        print("JSON 文件格式错误，请检查文件内容是否为有效的 JSON。")
    except Exception as e:
        print(f"发生了一个错误: {e}")

# 调用函数
json_file = "player1.json"  # 替换为你的 JSON 文件路径
inspect_player_fields(json_file)


player 字段下的所有键：
- lastUpdated
- attack
- possession
- carries
- defending
- goalkeeping


In [12]:
#保存进攻数据
def save_attack(json_file):
    try:
        # 打开并读取 JSON 文件
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # 获取 player 信息
        players = data["player"]["attack"]["overall"]
        
        # 用于存储所有球员信息的列表
        info_list = []
        
        for player_data in players:
            player = {
                "Player ID": player_data["player_id"],   # 球员ID
                "Name": player_data["player"],  # 球员姓名
                "Date of Birth": player_data["date_of_birth"],  # 球员出生日期
                "Age": player_data["age"],   # 球员年龄
                "Team": player_data["contestantName"],   # 球员所在队伍名字
                "Shirt Number": player_data["shirt_number"],    # 球员球衣号码
                "Squad Position": player_data["squad_position"],   # 球员位置
                "Apps": player_data["apps"],  # 球员上场次数
                "Minutes Played": player_data["mins_played"],  # 球员上场时间
                "Goals": player_data["goals"],  # 进球数
                "xG": player_data["xg"], # 期望进球数
                "Goals vs xG": player_data["goals_vs_xg"], # 实际进球与期望进球的差
                "Shots": player_data["shots"],  # 射门数
                "Shots on Target": player_data["shots_on_target"], # 射正的次数
                "Shot Conversion": player_data["shot_conv"],  # 射门转化率
                "xG per Shot": player_data["xg_per_shot"]  # 每次射门的期望进球数
            }
            info_list.append(player)
        
        # 使用 pandas 创建 DataFrame
        df = pd.DataFrame(info_list)
        
        # 保存为 Excel 文件
        df.to_excel("players_attack_data.xlsx", index=False)
        print("数据已经保存为表格")

        return True  # 成功执行后返回 True

    except FileNotFoundError:
        print(f"文件 {json_file} 未找到，请检查文件路径。")
    except json.JSONDecodeError:
        print("JSON 文件格式错误，请检查文件内容是否为有效的 JSON。")
    except Exception as e:
        print(f"发生了一个错误: {e}")
    
    return False  # 如果发生错误，返回 False

In [ ]:
save_attack("player.json")

In [32]:
#保存助攻数据
def save_creation(json_file):
    try:
        # 打开并读取 JSON 文件
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # 获取 player 信息
        players = data["player"]["possession"]["chanceCreation"]
        
        # 用于存储所有球员信息的列表
        info_list = []
        
        for player_data in players:
            player = {
                "Player ID": player_data["player_id"],   # 球员ID
                "Name": player_data["player"],  # 球员姓名
                "Date of Birth": player_data["date_of_birth"],  # 球员出生日期
                "Age": player_data["age"],   # 球员年龄
                "Team": player_data["contestantName"],   # 球员所在队伍名字
                "Shirt Number": player_data["shirt_number"],    # 球员球衣号码
                "Squad Position": player_data["squad_position"],   # 球员位置
                "Apps": player_data["apps"],  # 球员上场次数
                "Minutes Played": player_data["mins_played"],  # 球员上场时间
                "chances created": player_data["chances_created"],  # 机会创造
                "xA": player_data["xa"], # 期望助攻数
                "assists": player_data["assists"], # 助攻数
                "op_assists": player_data["op_assists"],  # 为对手创造助攻数
            }
            info_list.append(player)
        
        # 使用 pandas 创建 DataFrame
        df = pd.DataFrame(info_list)
        
        # 保存为 Excel 文件
        df.to_excel("players_chanceCreation_data.xlsx", index=False)
        print("数据已经保存为表格")

        return True  # 成功执行后返回 True

    except FileNotFoundError:
        print(f"文件 {json_file} 未找到，请检查文件路径。")
    except json.JSONDecodeError:
        print("JSON 文件格式错误，请检查文件内容是否为有效的 JSON。")
    except Exception as e:
        print(f"发生了一个错误: {e}")
    
    return False  # 如果发生错误，返回 False

In [34]:
save_creation("player1.json")

数据已经保存为表格


True

In [28]:
#保存过人数据
def save_passing(json_file):
    try:
        # 打开并读取 JSON 文件
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # 获取 player 信息
        players = data["player"]["possession"]["passing"]
        
        # 用于存储所有球员信息的列表
        info_list = []
        
        for player_data in players:
            player = {
                "Player ID": player_data["player_id"],   # 球员ID
                "Name": player_data["player"],  # 球员姓名
                "Date of Birth": player_data["date_of_birth"],  # 球员出生日期
                "Age": player_data["age"],   # 球员年龄
                "Team": player_data["contestantName"],   # 球员所在队伍名字
                "Shirt Number": player_data["shirt_number"],    # 球员球衣号码
                "Squad Position": player_data["squad_position"],   # 球员位置
                "Apps": player_data["apps"],  # 球员上场次数
                "Minutes Played": player_data["mins_played"],  # 球员上场时间
                "passes": player_data["passes"],  # 总共传球次数
                "pass perc": player_data["pass_perc"], # 传球成功率
                "successful passes": player_data["successful_passes"], # 传球成功次数
                "successful_final_third_passes": player_data["successful_final_third_passes"],  # 最后三分之一传球成功率
                
            }
            info_list.append(player)
        
        # 使用 pandas 创建 DataFrame
        df = pd.DataFrame(info_list)
        
        # 保存为 Excel 文件
        df.to_excel("players_passing_data.xlsx", index=False)
        print("数据已经保存为表格")

        return True  # 成功执行后返回 True

    except FileNotFoundError:
        print(f"文件 {json_file} 未找到，请检查文件路径。")
    except json.JSONDecodeError:
        print("JSON 文件格式错误，请检查文件内容是否为有效的 JSON。")
    except Exception as e:
        print(f"发生了一个错误: {e}")
    
    return False  # 如果发生错误，返回 False

In [103]:
save_passing("player1.json")

数据已经保存为表格


True

In [24]:
#保存防守数据
def save_defending(json_file):
    try:
        # 打开并读取 JSON 文件
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # 获取 player 信息
        players = data["player"]["defending"]["overall"]
        
        # 用于存储所有球员信息的列表
        info_list = []
        
        for player_data in players:
            player = {
                "Player ID": player_data["player_id"],   # 球员ID
                "Name": player_data["player"],  # 球员姓名
                "Date of Birth": player_data["date_of_birth"],  # 球员出生日期
                "Age": player_data["age"],   # 球员年龄
                "Team": player_data["contestantName"],   # 球员所在队伍名字
                "Shirt Number": player_data["shirt_number"],    # 球员球衣号码
                "Squad Position": player_data["squad_position"],   # 球员位置
                "Apps": player_data["apps"],  # 球员上场次数
                "Minutes Played": player_data["mins_played"],  # 球员上场时间
                "tackles": player_data["tackles"],  #  铲球
                "interceptions": player_data["interceptions"], # 拦截
                "recoveries": player_data["recoveries"], # 恢复球权
                "blocks": player_data["blocks"],  # 封堵
                "clearances": player_data["clearances"], #解围
                "ground_duel_perc": player_data["ground_duel_perc"], #地面对抗胜利 
                "aerial_dual_perc": player_data["aerial_dual_perc"], #空中对抗胜利
                
                
            }
            info_list.append(player)
        
        # 使用 pandas 创建 DataFrame
        df = pd.DataFrame(info_list)
        
        # 保存为 Excel 文件
        df.to_excel("players_defending_data.xlsx", index=False)
        print("数据已经保存为表格")

        return True  # 成功执行后返回 True

    except FileNotFoundError:
        print(f"文件 {json_file} 未找到，请检查文件路径。")
    except json.JSONDecodeError:
        print("JSON 文件格式错误，请检查文件内容是否为有效的 JSON。")
    except Exception as e:
        print(f"发生了一个错误: {e}")
    
    return False  # 如果发生错误，返回 False

In [26]:
save_defending("player1.json")

数据已经保存为表格


True

In [54]:
# 读取合并后的数据文件
file_path = 'player_data.xlsx'
df = pd.read_excel(file_path)

# 第一步：清除含有缺失值的行
df_cleaned = df.dropna()

# 第二步：删除至少 5 列数据值为 0 或 0.00 的行
# 统计每行中值为 0 的列数
zero_count_per_row = (df_cleaned == 0).sum(axis=1)

# 筛选出满足条件（>= 5 列值为 0）的行索引
rows_to_remove = zero_count_per_row[zero_count_per_row >= 5].index

# 删除这些行
df_final = df_cleaned.drop(index=rows_to_remove)

# 保存清理后的结果
df_final.to_excel('cleaned_player_data.xlsx', index=False)

print("数据清理完成，结果已保存为 cleaned_player_data.xlsx！")


数据清理完成，结果已保存为 cleaned_player_data.xlsx！


In [44]:
print(attack_df.columns)
print(creation_df.columns)
print(passing_df.columns)
print(defending_df.columns)


Index(['Player ID', 'Name', 'Date of Birth', 'Age', 'Team', 'Shirt Number',
       'Squad Position', 'Apps', 'Minutes Played', 'Goals', 'xG',
       'Goals vs xG', 'Shots', 'Shots on Target', 'Shot Conversion',
       'xG per Shot'],
      dtype='object')
Index(['Player ID', 'Name', 'Date of Birth', 'Age', 'Team', 'Shirt Number',
       'Squad Position', 'Apps', 'Minutes Played', 'chances created', 'xA',
       'assists', 'op_assists'],
      dtype='object')
Index(['Player ID', 'Name', 'Date of Birth', 'Age', 'Team', 'Shirt Number',
       'Squad Position', 'Apps', 'Minutes Played', 'passes', 'pass perc',
       'successful passes', 'successful_final_third_passes'],
      dtype='object')
Index(['Player ID', 'Name', 'Date of Birth', 'Age', 'Team', 'Shirt Number',
       'Squad Position', 'Apps', 'Minutes Played', 'tackles', 'interceptions',
       'recoveries', 'blocks', 'clearances', 'ground_duel_perc',
       'aerial_dual_perc'],
      dtype='object')


In [48]:
print(set(attack_df['Player ID']) & set(creation_df['Player ID']) & set(passing_df['Player ID']) & set(defending_df['Player ID']))
print(merged_df.isnull().sum())

{448514, 499717, 229384, 499721, 178186, 235530, 454667, 499724, 204814, 499726, 149519, 501770, 184341, 184349, 219168, 475168, 491557, 460842, 444463, 532529, 231480, 204863, 184386, 206915, 194634, 501837, 200785, 153682, 596054, 215136, 223340, 544877, 106606, 424044, 219249, 106611, 227444, 178301, 209036, 219279, 110735, 209042, 469142, 209046, 516250, 147611, 592031, 55459, 245923, 204968, 176297, 463034, 57531, 590012, 577725, 438464, 577731, 641221, 55494, 223434, 213198, 198869, 221399, 463067, 547037, 467169, 461025, 497894, 481510, 489706, 514287, 477424, 202993, 39155, 473341, 106760, 80146, 221466, 432422, 518438, 514356, 223541, 551226, 551230, 231747, 510281, 244042, 215379, 536916, 207189, 250199, 481624, 209243, 209244, 444765, 184667, 463212, 428399, 215413, 498046, 209289, 477580, 215439, 86417, 510362, 510363, 487838, 197024, 88484, 172453, 149929, 156074, 455084, 141746, 184754, 207283, 577974, 543158, 154048, 221632, 465351, 18892, 512462, 502222, 59859, 444884, 